# Educación Continua

Se importan librerías de pandas, numpy, matplotlib y seabron; de minisom, se importó el módulo MiniSom, del módulo sklearn.cluster se importó la clase KMeans y del módulo sklearn.metrics, se importó pairwise_distances_argmin_min.

%matplotlib inline es una 'magic function' de iPython. El propósito de esta línea de código es que los plots de matplotlib se desplieguen dentro de la misma celda y justo debajo del código que llama a la librería. plt.rcParams se utiliza para controlar el tamaño de las gráficas. plt.style.use es para seleccionar un estilo de gráficas

In [ ]:
#By José Francisco Pacheco Quintana a.k.a. 'pach'
#Importing libraries needed for data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import pprint
import c45

from minisom import MiniSom
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

 
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (30, 20) 
plt.rc('font', size=20) 
#Determines the size of the graphs
plt.style.use('ggplot')

Se creó un dataframe que consiste con los datos obtenidos de las ECOAS. Se eliminaron ciertas columnas que se cree que son irrelevantes para el análisis de la información (además de que se libera espacio de memoria y facilita el procesamiento): se descartó comentario porque ya se tiene el puntaje del comentario; correo_coordinador y correo_instructor porque ya se tiene una columna con los nombres de los coordinadores e instructores por lo que esta información sería un tanto redundante para el análisis.

In [ ]:
#Getting a dataframe from excel file
df = pd.read_excel (r'comments_final.xlsx')
#Getting rid of unuseful data
clean_data = df.drop(['comentario', 'correo_coordinador', 'correo_instructor'], axis = 1)

Se creó un dataframe que consiste únicamente en las columnas nps y puntaje_sentimiento. Se borraron los campos que tenían datos que contienen campos vacíos, ya que algunos algoritmos no se puede aplicar a campos vacíos. Posteriormente se creó un histograma de las columnas nps y puntaje_sentimiento, para mostrar la distribución de los datos. Se puede observar en las tablas que la mayor parte de los registros tienen un nps en el percentil más alto, mientras que para puntaje_sentimiento, se puede observar que la mayor parte de los registros tienen un puntaje neutral.

In [ ]:
#Creating a dataframe with relevant numerical values: nps and puntaje_sentimiento
data_km = clean_data[['puntaje_sentimiento', 'nps']].copy()
data_km = data_km.dropna()
#Creating an histogram for showing the distribution of puntaje_sentimiento agains nps
data_km.hist()

Se creó un array X de dos dimensiones con las columnas del dataframe data_km. Posteriormente, se introdujo el array a una función del módulo KMeans, con el propósito de determinar el número de clusters en los que se puede agrupar los registros introducidos. Se utilizó el método de evaluación de distancias entre los datos y los centroides. En la gráfica, score corresponde a esta distancia. Se puede observar que cuando la curva llega a 5, se produce un 'codo', indicando que este es un número de clusters ideal para los datos introducidos.

<h2>K-Means algorithm</h2>

In [ ]:
X = np.array(data_km[["puntaje_sentimiento", "nps"]])

In [ ]:
#Getting the number of clusters
Nc = range(1, 20)
kmeans = [KMeans(n_clusters=i) for i in Nc]
kmeans
score = [kmeans[i].fit(X).score(X) for i in range(len(kmeans))]
score
plt.plot(Nc,score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

Se aplica el algoritmo K-Means a las columnas de nps y puntaje_sentimiento. El algoritmo de K-means clasifica los datos introducidos midiendo las distancias euclidianas entre los registros y los centroides, realizando múltiples iteraciones hasta que se llega a una en la cual las distancias sean óptimas. También se obtienen los centroides de cada cluster.

In [ ]:
#Applying K-Means algorithm to the nps and puntaje_sentimiento columns from the data
kmeans = KMeans(n_clusters=5).fit(data_km)
centroids = kmeans.cluster_centers_
print(centroids)

plt.scatter(data_km['puntaje_sentimiento'], data_km['nps'], c= kmeans.labels_.astype(float), s=50, alpha=0.5)
plt.scatter(centroids[:, 0], centroids[:, 1], c='red', s=50)


Se busca entre los registros aquellos que son más cercanos a cada uno de los centroides.

In [ ]:
#Obtaining the id of the registers that are the closest to each centroid
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, data_km)
closest

In [ ]:
cluster_map = pd.DataFrame()
cluster_map['data_index'] = data_km.index.values
cluster_map['cluster'] = kmeans.labels_

new_column = cluster_map[['cluster']]

merged = clean_data.merge(new_column, left_index=True, right_index=True, how='inner')

In [ ]:
from pandas import ExcelWriter
writer = ExcelWriter('comments_cluster.xlsx')
merged.to_excel(writer,'Sheet1')
writer.save()

In [ ]:
merged.dropna()
dict_of_cluster = {k: v for k, v in merged.groupby('cluster')}

In [ ]:
df_clust = dict_of_cluster[0]

dict_by_campus = {k: v for k, v in df_clust.groupby('campus')}

dict_by_inst = {k: v for k, v in df_clust.groupby('instructor')}


Ejemplo de estadísticas por campus:

In [ ]:
A = dict_by_campus['AGS (Aguascalientes)']

campus_mode = A.drop(['categoria', 'subcategoria', 'fecha_coordinador', 'campus', 'cluster', 'id'], axis = 1)

print(campus_mode.mode())
print(A.mean(axis = 0, skipna = True) )

Ejemplo de estadísticas por instructor

In [ ]:
B = dict_by_inst['ANA LOURDES LOPEZ MONDRAGON']

inst_mode = B.drop(['categoria', 'subcategoria', 'fecha_coordinador', 'campus', 'cluster', 'instructor', 'id'], axis = 1)

print(inst_mode.mode())
print(B.mean(axis = 0, skipna = True) )

<h3> Análisis por grupo </h3>

In [ ]:
group_data = pd.read_excel (r'comments_grupo.xlsx')
dict_by_group = {k: v for k, v in group_data.groupby('grupo')}

Grupo 2:
Campus Ciudad Juárez y Campus Central de Veracruz

Campus Ciudad Juárez:

In [ ]:
group_data2 = dict_by_group[2]
dict_by_camp = {k: v for k, v in group_data.groupby('campus')}
camp_cdj = dict_by_camp['CDJ (Cd. Juárez)']

low = camp_cdj.loc[(camp_cdj['nps'] < 75)]
mode = low.drop(['id', 'correo_instructor', 'grupo'], axis = 1)

print('modas: ')
print('')
print(mode.mode())
print('')
print('promedios: ')
print('')
print(low.mean(axis = 0, skipna = True))
print('')
print('desviación estándar: ')
print('')
print(low.std(axis = 0, skipna = True))

In [ ]:
#def estadisticas(campus)

I = low['instructor'].unique()
for x in I:
    inst = camp_cdj.loc[(camp_cdj['instructor'] == x)]
    inst_est = inst.drop(['id', 'correo_instructor', 'grupo', 'campus', 'id_programa', 'id_modulo'], axis = 1)
    print('promedios de ' + x)
    print('')
    print(inst_est.mean(axis = 0, skipna = True))
    print('')
    print('desviaciones estándar de  ' + x)
    print('')
    print(inst_est.std(axis = 0, skipna = True))
    print('')
    print('medianas de  ' + x)
    print('')
    print(inst_est.median(axis = 0, skipna = True))
    print('______________________________________________________________________')

In [ ]:
print(low['instructor'].unique())


Grupo 3: Campus Ciudad Obregón, Campus Cuernavaca, Campus Hidalgo y Campus Irapuato

In [ ]:
group_data3 = dict_by_group[3]
print(group_data3['campus'].unique())

<h2>Naïve Bayes Algorithm (Development)</h2>

In [ ]:
df = pd.read_table('SMSSpamCollection',
                   sep='\t', 
                   header=None,
                   names=['label', 'message'])

In [ ]:
df['label'] = df.label.map({'ham': 0, 'spam': 1})

In [ ]:
df['message'] = df.message.map(lambda x: x.lower())
df['message'] = df.message.str.replace('[^\w\s]', '')

In [ ]:
import nltk
nltk.download()

In [ ]:
df['message'] = df['message'].apply(nltk.word_tokenize)

In [ ]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
 
df['message'] = df['message'].apply(lambda x: [stemmer.stem(y) for y in x])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# This converts the list of words into space-separated strings
df['message'] = df['message'].apply(lambda x: ' '.join(x))

count_vect = CountVectorizer()
counts = count_vect.fit_transform(df['message'])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(counts, df['label'], test_size=0.1, random_state=69)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB().fit(X_train, y_train)
print(model)
print(counts)

In [ ]:
#Setting up data

merged = merged.dropna()
NB_data = merged[['tipo', 'campus', 'id_programa', 'nps', 'id_modulo', 'instructor', 'nombre_coordinador', 'indicador', 'categoria', 'subcategoria']].copy()
NB_data['nps75'] = (NB_data['nps'] >= 75)
NB_data = NB_data.drop(['nps'], axis = 1)
NB_data["id_programa"]= NB_data["id_programa"].astype(str)
NB_data["id_modulo"]= NB_data["id_modulo"].astype(str)
NB_data["nps75"]= NB_data["nps75"].astype(str)

In [ ]:
test = NB_data[['tipo', 'nps75']].copy()
counts = test.values
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(counts, test[['tipo', 'nps75'] ], test_size=0.1, random_state=69)

from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB().fit(X_train, y_train)

<h2>1R Algorithm (Development)</h2>

In [ ]:
test = pd.read_excel (r'Test.xlsx')
print(test)

In [ ]:
#converting the types of the columns to strings and booleans
print(test.dtypes)


for col in test.columns:
    if(test[col].dtype == np.object or test[col].dtype == np.bool):
          pass
    else:
          test[col] = test[col].astype(str)
print(test.dtypes)

In [ ]:
from fractions import Fraction

#print(test['Outlook'])
a = test['Outlook'].value_counts()
target = test['Play'].value_counts()
#length_target = target.size
#length = a.size
x = 0
gkk = test.groupby(['Outlook'])
print(gkk)


"""while x < length:
    numerator = a.iloc[x]
    print(a.iloc[x])
    
    x = x+1 
"""
#if a(sunny) > a.iloc[:2]:
 
    #print(a[:0])
#else:
 #   pass
#print(a)

<h2>CART Algorithm (Development)</h2>

In [ ]:
#Testing with sample dataset
import decision_tree as dt
dec_tree = test.values #converting the dataframe into an array
my_tree = dt.build_tree(dec_tree)
dt.print_tree(my_tree)

In [ ]:
from pandas import ExcelWriter
writer = ExcelWriter('nps75.xlsx')
NB_data.to_excel(writer,'Sheet1')
writer.save()

In [ ]:
nps75 = pd.read_excel (r'nps75.xlsx')

for col in nps75:
    nps75[col] = [np.nan if (not isinstance(val, str) and np.isnan(val)) else 
               (val if isinstance(val, str) else str(int(val))) 
               for val in nps75[col].tolist()]

print(nps75.dtypes)

In [ ]:
#Using the dataframe
import decision_tree as dt
dec_tree = nps75.values #converting the dataframe into an array
my_tree = dt.build_tree(dec_tree)
dt.print_tree(my_tree)